# Building a custom hg38/WSN genome

My RNA seq examining 5'ppp RNA will return both human and influenza virus RNAs. I need to combine the human and WSN genomes so that I can align to both. This is more efficient than aligning to each separately and will give a better idea of coverage of both genomes.

## Annotation

The human genome is well annotated and hg38 comes with a GCF file. This file gives coordinates and descriptions for exons, genes, and transcripts in the corresponding genome assembly file. Aligners like `STAR` and `bowtie2` use this file when indexing genomes prior to alignment to 